In [ ]:
!pwd
!ls

In [ ]:
TRACK_DEGRADATION_FACTORS = {
    "Monaco": 0.7,
    "Monza": 0.8,
    "Silverstone": 1.0,
    "Suzuka": 1.2,
    "Hungary": 1.3
}

In [ ]:
DRIVER_STYLE_FACTORS = {
    "Aggressive": 1.1,
    "Balanced": 1.0,
    "TyreSaver": 0.9
}

In [ ]:
import numpy as np
np.random.seed(42)

In [ ]:
import sys
sys.path.append("src")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from data_loader import load_race_data
from degradation import TyreDegradationModel
from strategy import one_stop_strategy, two_stop_strategy

In [ ]:
def create_track_aware_model(compound, track):
    model = TyreDegradationModel(compound=compound)
    track_factor = TRACK_DEGRADATION_FACTORS.get(track, 1.0)
    model.alpha *= track_factor
    return model

In [ ]:
race_data = load_race_data("data/processed/race_data.csv")
race_data.head()

In [ ]:
import inspect
from degradation import TyreDegradationModel
print(inspect.signature(TyreDegradationModel.predict))

In [ ]:
model = TyreDegradationModel(compound="Medium")
model.fit(race_data)

In [ ]:
tyre_life = np.arange(1, 31)
curve = model.predict(tyre_life)

plt.figure()
plt.plot(tyre_life, curve)
plt.xlabel("Tyre Life (laps)")
plt.ylabel("Relative Performance")
plt.title("Tyre Degradation vs Tyre Life")
plt.show()

In [ ]:
tyre_life = np.arange(1, 31)

compounds = ["Soft", "Medium", "Hard"]

plt.figure(figsize=(8,5))

for compound in compounds:
    model = TyreDegradationModel(compound=compound)
    curve = model.predict(tyre_life)
    plt.plot(tyre_life, curve, label=compound)

plt.xlabel("Tyre Life (laps)")
plt.ylabel("Relative Performance")
plt.title("Tyre Degradation by Compound")
plt.legend()
plt.show()

In [ ]:
from strategy import one_stop_strategy

total_laps = 70
model = TyreDegradationModel(compound="Medium")

pit_range = range(15, 41)
one_stop_results = []

for pit in pit_range:
    cost = one_stop_strategy(model, total_laps, pit)
    one_stop_results.append((pit, cost))

one_stop_df = pd.DataFrame(one_stop_results, columns=["Pit Lap", "Total Degradation Cost"])
one_stop_df.head()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(one_stop_df["Pit Lap"], one_stop_df["Total Degradation Cost"])
plt.xlabel("Pit Stop Lap")
plt.ylabel("Total Degradation Cost")
plt.title("One-Stop Strategy Cost (Medium Compound)")
plt.show()

In [ ]:
from strategy import two_stop_strategy

two_stop_results = []

for pit1 in range(15, 30):
    for pit2 in range(pit1 + 10, 50):
        cost = two_stop_strategy(model, total_laps, pit1, pit2)
        two_stop_results.append((pit1, pit2, cost))

two_stop_df = pd.DataFrame(
    two_stop_results,
    columns=["Pit 1", "Pit 2", "Total Degradation Cost"]
)

two_stop_df.sort_values("Total Degradation Cost").head()

In [ ]:
best_one_stop = one_stop_df.loc[
    one_stop_df["Total Degradation Cost"].idxmin()
]

best_one_stop

In [ ]:
best_two_stop = two_stop_df.loc[
    two_stop_df["Total Degradation Cost"].idxmin()
]

best_two_stop

In [ ]:
comparison_df = pd.DataFrame([
    ["One-stop", best_one_stop["Total Degradation Cost"]],
    ["Two-stop", best_two_stop["Total Degradation Cost"]],
], columns=["Strategy", "Total Degradation Cost"])

comparison_df

In [ ]:
compounds = ["Soft", "Medium", "Hard"]
total_laps = 70
pit_range = range(15, 41)

compound_one_stop_results = []

for compound in compounds:
    model = TyreDegradationModel(compound=compound)
    
    for pit in pit_range:
        cost = one_stop_strategy(model, total_laps, pit)
        compound_one_stop_results.append(
            (compound, pit, cost)
        )

compound_one_stop_df = pd.DataFrame(
    compound_one_stop_results,
    columns=["Compound", "Pit Lap", "Total Degradation Cost"]
)

compound_one_stop_df.head()

In [ ]:
best_one_stop_by_compound = (
    compound_one_stop_df
    .loc[compound_one_stop_df.groupby("Compound")["Total Degradation Cost"].idxmin()]
)

best_one_stop_by_compound

In [ ]:
plt.figure(figsize=(8,5))

for compound in compounds:
    subset = compound_one_stop_df[
        compound_one_stop_df["Compound"] == compound
    ]
    plt.plot(
        subset["Pit Lap"],
        subset["Total Degradation Cost"],
        label=compound
    )

plt.xlabel("Pit Stop Lap")
plt.ylabel("Total Degradation Cost")
plt.title("One-Stop Strategy Cost by Compound")
plt.legend()
plt.show()

In [ ]:
compound_two_stop_results = []

for compound in compounds:
    model = TyreDegradationModel(compound=compound)
    
    for pit1 in range(15, 30):
        for pit2 in range(pit1 + 10, 50):
            cost = two_stop_strategy(model, total_laps, pit1, pit2)
            compound_two_stop_results.append(
                (compound, pit1, pit2, cost)
            )

compound_two_stop_df = pd.DataFrame(
    compound_two_stop_results,
    columns=["Compound", "Pit 1", "Pit 2", "Total Degradation Cost"]
)

compound_two_stop_df.head()

In [ ]:
best_two_stop_by_compound = (
    compound_two_stop_df
    .loc[compound_two_stop_df.groupby("Compound")["Total Degradation Cost"].idxmin()]
)

best_two_stop_by_compound

In [ ]:
import numpy as np

def monte_carlo_strategy(
    strategy_fn,
    compound,
    total_laps,
    pit_params,
    alpha_mean,
    alpha_std,
    n_sim=1000
):
    costs = []

    for _ in range(n_sim):
        sampled_alpha = np.random.normal(alpha_mean, alpha_std)

        model = TyreDegradationModel(compound=compound)

        model.alpha = sampled_alpha

        cost = strategy_fn(model, total_laps, *pit_params)
        costs.append(cost)

    costs = np.array(costs)

    return {
        "mean_cost": costs.mean(),
        "std_cost": costs.std(),
        "ci_lower": np.percentile(costs, 2.5),
        "ci_upper": np.percentile(costs, 97.5)
    }

In [ ]:
mc_one_stop = monte_carlo_strategy(
    strategy_fn=one_stop_strategy,
    compound="Soft",
    total_laps=70,
    pit_params=(35,),
    alpha_mean=0.03,
    alpha_std=0.005,
    n_sim=2000
)

mc_two_stop = monte_carlo_strategy(
    strategy_fn=two_stop_strategy,
    compound="Soft",
    total_laps=70,
    pit_params=(23, 46),
    alpha_mean=0.03,
    alpha_std=0.005,
    n_sim=2000
)

mc_one_stop, mc_two_stop

In [ ]:
mc_one_stop_medium = monte_carlo_strategy(
    strategy_fn=one_stop_strategy,
    compound="Medium",
    total_laps=70,
    pit_params=(best_one_stop_by_compound.loc[
        best_one_stop_by_compound["Compound"] == "Medium", "Pit Lap"
    ].values[0],),
    alpha_mean=0.03,
    alpha_std=0.005,
    n_sim=2000
)

mc_two_stop_medium = monte_carlo_strategy(
    strategy_fn=two_stop_strategy,
    compound="Medium",
    total_laps=70,
    pit_params=(
        best_two_stop_by_compound.loc[
            best_two_stop_by_compound["Compound"] == "Medium", "Pit 1"
        ].values[0],
        best_two_stop_by_compound.loc[
            best_two_stop_by_compound["Compound"] == "Medium", "Pit 2"
        ].values[0],
    ),
    alpha_mean=0.03,
    alpha_std=0.005,
    n_sim=2000
)

mc_one_stop_medium, mc_two_stop_medium

In [ ]:
mc_one_stop_hard = monte_carlo_strategy(
    strategy_fn=one_stop_strategy,
    compound="Hard",
    total_laps=70,
    pit_params=(best_one_stop_by_compound.loc[
        best_one_stop_by_compound["Compound"] == "Hard", "Pit Lap"
    ].values[0],),
    alpha_mean=0.02,
    alpha_std=0.004,
    n_sim=2000
)

mc_two_stop_hard = monte_carlo_strategy(
    strategy_fn=two_stop_strategy,
    compound="Hard",
    total_laps=70,
    pit_params=(
        best_two_stop_by_compound.loc[
            best_two_stop_by_compound["Compound"] == "Hard", "Pit 1"
        ].values[0],
        best_two_stop_by_compound.loc[
            best_two_stop_by_compound["Compound"] == "Hard", "Pit 2"
        ].values[0],
    ),
    alpha_mean=0.02,
    alpha_std=0.004,
    n_sim=2000
)

mc_one_stop_hard, mc_two_stop_hard

In [ ]:
mc_summary = pd.DataFrame([
    ["Soft", "One-stop", mc_one_stop["mean_cost"], mc_one_stop["std_cost"],
     mc_one_stop["ci_lower"], mc_one_stop["ci_upper"]],
    ["Soft", "Two-stop", mc_two_stop["mean_cost"], mc_two_stop["std_cost"],
     mc_two_stop["ci_lower"], mc_two_stop["ci_upper"]],

    ["Medium", "One-stop", mc_one_stop_medium["mean_cost"], mc_one_stop_medium["std_cost"],
     mc_one_stop_medium["ci_lower"], mc_one_stop_medium["ci_upper"]],
    ["Medium", "Two-stop", mc_two_stop_medium["mean_cost"], mc_two_stop_medium["std_cost"],
     mc_two_stop_medium["ci_lower"], mc_two_stop_medium["ci_upper"]],

    ["Hard", "One-stop", mc_one_stop_hard["mean_cost"], mc_one_stop_hard["std_cost"],
     mc_one_stop_hard["ci_lower"], mc_one_stop_hard["ci_upper"]],
    ["Hard", "Two-stop", mc_two_stop_hard["mean_cost"], mc_two_stop_hard["std_cost"],
     mc_two_stop_hard["ci_lower"], mc_two_stop_hard["ci_upper"]],
],
columns=["Compound", "Strategy", "Mean Cost", "Std Dev", "CI Lower", "CI Upper"])

mc_summary

In [ ]:
plt.figure(figsize=(8,5))

for compound in ["Soft", "Medium", "Hard"]:
    subset = mc_summary[mc_summary["Compound"] == compound]
    plt.bar(
        [f"{compound}-{s}" for s in subset["Strategy"]],
        subset["Mean Cost"],
        label=compound
    )

plt.ylabel("Mean Degradation Cost")
plt.title("Monte Carlo Strategy Comparison Across Compounds")
plt.show()

In [ ]:
mc_summary.to_csv("monte_carlo_strategy_summary.csv", index=False)
mc_summary

In [ ]:
plt.figure(figsize=(9,5))

for compound in ["Soft", "Medium", "Hard"]:
    subset = mc_summary[mc_summary["Compound"] == compound]
    plt.plot(
        subset["Strategy"],
        subset["Mean Cost"],
        marker="o",
        label=compound
    )

plt.ylabel("Mean Degradation Cost")
plt.title("Monte Carlo Strategy Comparison Across Compounds")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
print("Monte Carlo analysis completed successfully.")
print("Results saved to monte_carlo_strategy_summary.csv")

Layer 2 code begins from here
Layer 2: Context-Aware Strategy Insights

Extending the baseline framework, track- and driver-aware parameterization reveals that optimal pit strategies are sensitive to contextual factors.
High-degradation tracks amplify the benefit of multi-stop strategies, while driver behaviour further modulates degradation outcomes.
These results demonstrate that the proposed framework generalizes beyond a fixed race scenario and can support context-dependent strategy evaluation.


In [ ]:
def monte_carlo_strategy_track_aware(
    strategy_fn,
    compound,
    track,
    total_laps,
    pit_params,
    alpha_std,
    n_sim=1000
):
    base_model = TyreDegradationModel(compound=compound)
    base_alpha = base_model.alpha * TRACK_DEGRADATION_FACTORS.get(track, 1.0)

    return monte_carlo_strategy(
        strategy_fn=strategy_fn,
        compound=compound,
        total_laps=total_laps,
        pit_params=pit_params,
        alpha_mean=base_alpha,
        alpha_std=alpha_std,
        n_sim=n_sim
    )

In [ ]:
track = "Monaco"

mc_soft_one_stop_monaco = monte_carlo_strategy_track_aware(
    strategy_fn=one_stop_strategy,
    compound="Soft",
    track=track,
    total_laps=70,
    pit_params=(35,),
    alpha_std=0.005,
    n_sim=2000
)

mc_soft_two_stop_monaco = monte_carlo_strategy_track_aware(
    strategy_fn=two_stop_strategy,
    compound="Soft",
    track=track,
    total_laps=70,
    pit_params=(23, 46),
    alpha_std=0.005,
    n_sim=2000
)

mc_soft_one_stop_monaco, mc_soft_two_stop_monaco

In [ ]:
track = "Hungary"

mc_soft_one_stop_hungary = monte_carlo_strategy_track_aware(
    strategy_fn=one_stop_strategy,
    compound="Soft",
    track=track,
    total_laps=70,
    pit_params=(35,),
    alpha_std=0.005,
    n_sim=2000
)

mc_soft_two_stop_hungary = monte_carlo_strategy_track_aware(
    strategy_fn=two_stop_strategy,
    compound="Soft",
    track=track,
    total_laps=70,
    pit_params=(23, 46),
    alpha_std=0.005,
    n_sim=2000
)

mc_soft_one_stop_hungary, mc_soft_two_stop_hungary

In [ ]:
track_summary = pd.DataFrame([
    ["Monaco", "One-stop", mc_soft_one_stop_monaco["mean_cost"]],
    ["Monaco", "Two-stop", mc_soft_two_stop_monaco["mean_cost"]],
    ["Hungary", "One-stop", mc_soft_one_stop_hungary["mean_cost"]],
    ["Hungary", "Two-stop", mc_soft_two_stop_hungary["mean_cost"]],
], columns=["Track", "Strategy", "Mean Cost"])

track_summary

In [ ]:
def create_context_aware_model(compound, track, driver_style):
    model = TyreDegradationModel(compound=compound)

    model.alpha *= TRACK_DEGRADATION_FACTORS.get(track, 1.0)
    model.alpha *= DRIVER_STYLE_FACTORS.get(driver_style, 1.0)

    return model

In [ ]:
def monte_carlo_strategy_context_aware(
    strategy_fn,
    compound,
    track,
    driver_style,
    total_laps,
    pit_params,
    alpha_std,
    n_sim=1000
):
    base_model = create_context_aware_model(compound, track, driver_style)
    base_alpha = base_model.alpha

    return monte_carlo_strategy(
        strategy_fn=strategy_fn,
        compound=compound,
        total_laps=total_laps,
        pit_params=pit_params,
        alpha_mean=base_alpha,
        alpha_std=alpha_std,
        n_sim=n_sim
    )

In [ ]:
track = "Hungary"

mc_aggressive = monte_carlo_strategy_context_aware(
    strategy_fn=two_stop_strategy,
    compound="Soft",
    track=track,
    driver_style="Aggressive",
    total_laps=70,
    pit_params=(23, 46),
    alpha_std=0.005,
    n_sim=2000
)

mc_tyre_saver = monte_carlo_strategy_context_aware(
    strategy_fn=two_stop_strategy,
    compound="Soft",
    track=track,
    driver_style="TyreSaver",
    total_laps=70,
    pit_params=(23, 46),
    alpha_std=0.005,
    n_sim=2000
)

mc_aggressive, mc_tyre_saver

In [ ]:
driver_summary = pd.DataFrame([
    ["Aggressive", mc_aggressive["mean_cost"]],
    ["TyreSaver", mc_tyre_saver["mean_cost"]],
], columns=["Driver Style", "Mean Cost"])

driver_summary